In [1]:
import numpy as np
import random as rd
import matplotlib.pyplot as plt
from tqdm import tqdm

from fdtd_solver import FDTDSolver
from RF_index import RFIndex

In [ ]:
class SESimulation:
    def __init__(self, layer_materials, layer_thicknesses, solver):

        self.layer_materials = layer_materials
        self.layer_thicknesses = layer_thicknesses
        self.fdtd_solver = solver

        # Define frequency vectors for simulation
        self.f_vector = np.linspace(8e9, 18e9, 100)

    def perform_stackrt_simulation(self, n_matrix, f_vector):
        return self.fdtd_solver.stackrt(n_matrix, self.layer_thicknesses, f_vector, 0)

    def calculate_SE(self, RT):
        Ts = RT['Ts']
        Tp = RT['Tp']
        SE_Ts = -10 * np.log10(Ts)
        SE_Tp = -10 * np.log10(Tp)
        
        SE_Ts_mean = np.mean(SE_Ts)
        SE_Tp_mean = np.mean(SE_Tp)

        SE = (SE_Tp_mean + SE_Ts_mean) / 2
        return SE

    def run_simulation(self):
        # Create refractive index matrix
        n_matrix = RFIndex.create_matrix(self.fdtd_solver, self.layer_materials, self.f_vector)

        # Perform stackrt simulation
        RT = self.perform_stackrt_simulation(n_matrix, self.f_vector)

        # Calculate Spectral Efficiency (SE)
        SE = self.calculate_SE(RT)

        # Output results
        # print(f'stackrt: SE {SE:.4f}')
        return SE

In [ ]:
solver = FDTDSolver()

In [ ]:
material_database = {
    0: "Air",
    1: "TiO2 (Titanium Dioxide) - Siefke",
    2: "Ni (Nickel) - Palik",
    3: "SiO2 (Glass) - Palik",
    4: "Si3N4 (Silicon Nitride) - Luke",
    5: "Pd (Palladium) - Palik",
    6: "Cr (Chromium) - CRC",
    7: "Ag (Silver) - CRC",
    8: "Al (Aluminium) - CRC",
    9: "Ti (Titanium) - Palik",
    10: "W (Tungsten) - CRC",
    11: "Al2O3 - Palik",
    12: "TiN - Palik"
}

material_limits = {
    0: 0.0,
    1: 400e-9,
    2: 400e-9,
    3: 400e-9,
    4: 400e-9,
    5: 400e-9,
    6: 400e-9,
    7: 400e-9,
    8: 400e-9,
    9: 400e-9,
    10: 400e-9,
    11: 400e-9,
    12: 400e-9
}


def random_value(layer_code):
    layer_materials = []
    layer_thicknesses = []

    for code in layer_code:
        layer_materials.append(material_database[code])
        layer_thicknesses.append(rd.random() * material_limits[code])

    layer_thicknesses = np.array(layer_thicknesses)

    simulation = SESimulation(layer_materials, layer_thicknesses, solver)
    SE = simulation.run_simulation()
    
    return SE, layer_thicknesses

In [ ]:
def random_distribution(layer_code):
    N = 10000
    SEs = []
    max_SE = 0.0
    max_layer_thicknesses = np.array(len(layer_code))

    for i in range(N):
        SE, layer_thicknesses= random_value(layer_code)
        SEs.append(SE)
        if SE > max_SE:
            max_SE = SE
            max_layer_thicknesses = layer_thicknesses

    plt.hist(SEs, bins=100)
    plt.show()
    
    print(max_SE, layer_code, max_layer_thicknesses)
    
    return max_SE, max_layer_thicknesses

In [ ]:
def generate_sequence(length):
    sequence = [rd.randint(0, 12)]
    for _ in range(length - 2):
        choices = [x for x in range(1, 13) if x != sequence[-1]]
        sequence.append(rd.choice(choices))
    sequence.append(rd.randint(0, 12))
    
    return sequence

max_SEs = []
max_max_SE = 0.0

layer_number = 8
N_trials = 30

for i in tqdm(range(N_trials)):
    layer_code = generate_sequence(layer_number)
    max_max_layer_thicknesses = np.array(len(layer_code))

    max_SE, max_layer_thicknesses = random_distribution(layer_code)
    max_SEs.append(max_SE)
    
    if max_SE > max_max_SE:
        max_max_SE = max_SE
        max_max_layer_thicknesses = max_layer_thicknesses
        max_layer_code = layer_code
    
plt.hist(max_SEs, bins=100)
plt.show()

print(max_max_SE, max_layer_code, max_max_layer_thicknesses)